In [1]:
# Obtaining EEGEyeNet's benchmark framework
!git clone https://github.com/ardkastrati/EEGEyeNet.git
# Obtaining Our model implementations
!git clone https://github.com/AmCh-Q/CSCI6907Project.git
!cp -l CSCI6907Project/our_models.py EEGEyeNet
%cd EEGEyeNet

Cloning into 'EEGEyeNet'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 302 (delta 104), reused 88 (delta 88), pack-reused 180
Receiving objects: 100% (302/302), 84.92 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (165/165), done.
Cloning into 'CSCI6907Project'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), done.
Resolving deltas: 100% (2/2), done.
/content/EEGEyeNet


In [2]:
# Obtaining the datasets
!mkdir ./data
!wget -O ./data/Position_task_with_dots_synchronised_min.npz https://osf.io/download/ge87t/
!wget -O ./data/Position_task_with_dots_synchronised_min_hilbert.npz https://osf.io/download/bmrn9/

--2023-12-04 04:19:05--  https://osf.io/download/ge87t/
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/ktv7m/providers/dropbox/prepared/Position_task_with_dots_synchronised_min.npz [following]
--2023-12-04 04:19:05--  https://files.osf.io/v1/resources/ktv7m/providers/dropbox/prepared/Position_task_with_dots_synchronised_min.npz
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11075939748 (10G) [application/octet-stream]
Saving to: ‘./data/Position_task_with_dots_synchronised_min.npz’

./data/Position_tas 100%[===================>]  10.31G  13.4MB/s    in 11m 24s 

2023-12-04 04:30:32 (15.5 MB/s) - ‘./data/Position_task_with_dots_synchronised_min.npz’ saved [11075939748/11075939748]

--2023-12-04 04:3

In [3]:
# Configuring task, and dataset
# Modified from https://github.com/ardkastrati/EEGEyeNet
from config import config, build_file_name

config['task'] = 'Position_task'
config['dataset'] = 'dots'
config['preprocessing'] = 'min' # or max
config['feature_extraction'] = True # must be set to True for ML_models operating on feature extracted data
config['include_ML_models'] = True
config['include_DL_models'] = True
config['include_your_models'] = False
config['all_EEG_file'] = build_file_name()

In [4]:
# Add our model and SOTA
import benchmark
from hyperparameters import merge_models
from our_models import Ours_pretrained, EEGViT_pretrained
additional_models = {
    'Position_task': {
        'dots' : {
            'min_temp' : {
                'EEGViT': EEGViT_pretrained,
                'Ours': Ours_pretrained
            }
        }
    }
}
benchmark.all_models = merge_models(benchmark.all_models, additional_models)

In [ ]:
# Training and benchmarking the models
# Modified from https://github.com/ardkastrati/EEGEyeNet

# Google Colab Does not like tee-redirecting stdout (nothing gets printed)
# It also initialized its own logging configuration that must be overwritten
# So EEGEyeNet's main() needs to be modified here just for Colab
import sys
import time
import logging
from config import create_folder
from utils import IOHelper
from utils.tables_utils import print_table

# Setting up logging
create_folder()
logging.basicConfig(filename=config['info_log'], level=logging.INFO, force=True)
logging.info('Started the Logging')
logging.info(f"Using {config['framework']}")
start_time = time.time()

#Load the data
trainX, trainY = IOHelper.get_npz_data(config['data_dir'], verbose=True)

#Start benchmark
benchmark.benchmark(trainX, trainY)

logging.info("--- Runtime: %s seconds ---" % (time.time() - start_time))
logging.info('Finished Logging')

The results of the models can be found in directory ./runs/*/info.log